# Import Required Libraries
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
import mplfinance as mpf
import csv

In [ ]:
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
import mplfinance as mpf
import csv

# Configuration
last_close = 567.0
max_allowed = last_close * 1.10  # 623.7
num_days = 30

In [ ]:
# Configuration
last_close = 567.0
max_allowed = last_close * 1.10  # 623.7
num_days = 30

# Generate Timestamps
def generate_timestamps(start_time, num_days):
    current = datetime.utcfromtimestamp(start_time)
    timestamps = []
    days_added = 0
    while days_added < num_days:
        current += timedelta(days=1)
        if current.weekday() < 5:  # Skip weekends
            for i in range(13):  # 13 intervals/day (9:30 AM - 4:00 PM ET)
                interval = current.replace(hour=13, minute=30) + timedelta(minutes=30*i)
                timestamps.append(int(interval.timestamp()))
            days_added += 1
    return timestamps

In [ ]:
# Generate timestamps (UTC for NYSE trading hours)
def generate_timestamps(start_time, num_days):
    current = datetime.utcfromtimestamp(start_time)
    timestamps = []
    days_added = 0
    while days_added < num_days:
        current += timedelta(days=1)
        if current.weekday() < 5:  # Skip weekends
            for i in range(13):  # 13 intervals/day (9:30 AM - 4:00 PM ET)
                interval = current.replace(hour=13, minute=30) + timedelta(minutes=30*i)
                timestamps.append(int(interval.timestamp()))
            days_added += 1
    return timestamps

# Price Generator with Daily Bounds
def generate_ohlc(prev_close, daily_high, daily_low, num_intervals=13):
    opens = [prev_close]
    closes = []
    for _ in range(num_intervals-1):
        change = np.random.uniform(-0.002, 0.002)  # ±0.2% per 30m
        new_close = opens[-1] * (1 + change)
        new_close = np.clip(new_close, daily_low, daily_high)
        closes.append(new_close)
        opens.append(new_close)
    highs = [max(o, c) + np.random.uniform(0, 0.001)*o for o, c in zip(opens, closes)]
    lows = [min(o, c) - np.random.uniform(0, 0.001)*o for o, c in zip(opens, closes)]
    return list(zip(opens, highs, lows, closes))

In [ ]:
def generate_ohlc(prev_close, daily_high, daily_low, num_intervals=13):
    opens = [prev_close]
    closes = []
    for _ in range(num_intervals-1):
        change = np.random.uniform(-0.002, 0.002)  # ±0.2% per 30m
        new_close = opens[-1] * (1 + change)
        new_close = np.clip(new_close, daily_low, daily_high)
        closes.append(new_close)
        opens.append(new_close)
    highs = [max(o, c) + np.random.uniform(0, 0.001)*o for o, c in zip(opens, closes)]
    lows = [min(o, c) - np.random.uniform(0, 0.001)*o for o, c in zip(opens, closes)]
    return list(zip(opens, highs, lows, closes))

# Generate Data
timestamps = generate_timestamps(1742239800, num_days)
data = []
current_close = last_close
for ts in timestamps:
    if timestamps.index(ts) % 13 == 0:  # New day
        daily_high = min(current_close * 1.025, max_allowed)
        daily_low = max(current_close * 0.975, daily_high * 0.975)
    ohlc = generate_ohlc(current_close, daily_high, daily_low)
    for entry in ohlc:
        data.append([ts, entry[0], entry[1], entry[2], entry[3]])
        ts += 1800  # Add 30 mins
    current_close = ohlc[-1][3]

In [ ]:
# Generate Data
timestamps = generate_timestamps(1742239800, num_days)
data = []
current_close = last_close
for ts in timestamps:
    if timestamps.index(ts) % 13 == 0:  # New day
        daily_high = min(current_close * 1.025, max_allowed)
        daily_low = max(current_close * 0.975, daily_high * 0.975)
    ohlc = generate_ohlc(current_close, daily_high, daily_low)
    for entry in ohlc:
        data.append([ts, entry[0], entry[1], entry[2], entry[3]])
        ts += 1800  # Add 30 mins
    current_close = ohlc[-1][3]

# Convert Data to DataFrame
df = pd.DataFrame(data, columns=['time', 'open', 'high', 'low', 'close'])
df['time'] = pd.to_datetime(df['time'], unit='s')

In [ ]:
df = pd.DataFrame(data, columns=['time', 'open', 'high', 'low', 'close'])
df['time'] = pd.to_datetime(df['time'], unit='s')

# Plot the OHLC Data
mpf.plot(df.set_index('time'), type='ohlc', style='candlestick', title='Generated Stock Data', ylabel='Price')

In [ ]:
mpf.plot(df.set_index('time'), type='ohlc', style='classic', title='Generated Stock Data', ylabel='Price')

# Save to CSV
with open('generated_spy_30m.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['time', 'open', 'high', 'low', 'close'])
    writer.writerows(data)

In [ ]:
with open('generated_spy_30m.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['time', 'open', 'high', 'low', 'close'])
    writer.writerows(data)